# Strep and severity score correlations

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import biom
from biom import load_table
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import itertools
import matplotlib.patches
from scipy.stats import pearsonr



In [2]:
# Load the metadata
metadata_path = '../Data/Metadata/updated_clean_ant_skin_metadata.tab'
metadata = pd.read_csv(metadata_path, sep='\t')

metadata['#sample-id'] = metadata['#sample-id'].str.replace('_', '')
# Set Sample-ID as the index for the metadata dataframe 
metadata = metadata.set_index('#sample-id')


# Create group column based on case_type to simplify group names
metadata['group'] = metadata['case_type'].map({
    'case-lesional skin': 'skin-ADL',
    'case-nonlesional skin': 'skin-ADNL', 
    'control-nonlesional skin': 'skin-H',
    'case-anterior nares': 'nares-AD',
    'control-anterior nares': 'nares-H'
})

metadata

,PlateNumber,PlateLocation,i5,i5Sequence,i7,i7Sequence,identifier,Sequence,Plate ID,Well location,...,sex,enrolment_date,enrolment_season,hiv_exposure,hiv_status,household_size,o_scorad,FWD_filepath,REV_filepath,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1,A1,SA501,ATCGTACG,SA701,CGAGAGTT,SA701SA501,CGAGAGTT-ATCGTACG,1.010000e+21,A1,...,male,4/16/2015,Autumn,Unexposed,negative,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
900221,1,B1,SA502,ACTATCTG,SA701,CGAGAGTT,SA701SA502,CGAGAGTT-ACTATCTG,1.010000e+21,B1,...,female,8/11/2015,Winter,Unexposed,negative,7.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
Ca010EBL,1,C1,SA503,TAGCGAGT,SA701,CGAGAGTT,SA701SA503,CGAGAGTT-TAGCGAGT,1.010000e+21,C1,...,female,11/20/2014,Spring,Unexposed,negative,7.0,21,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
900460,1,D1,SA504,CTGCGTGT,SA701,CGAGAGTT,SA701SA504,CGAGAGTT-CTGCGTGT,1.010000e+21,D1,...,female,9/23/2015,Spring,Unexposed,NaN,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
900051,1,E1,SA505,TCATCGAG,SA701,CGAGAGTT,SA701SA505,CGAGAGTT-TCATCGAG,1.010000e+21,E1,...,male,4/21/2015,Autumn,Unexposed,negative,7.0,41,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca006ONL2,6,H1,SA508,GACACCGT,SB701,CTCGACTT,SB701SA508,CTCGACTT-GACACCGT,1.010000e+21,H1,...,female,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
Ca006ONNL,6,F2,SA506,CGTGAGTG,SB702,CGAAGTAT,SB702SA506,CGAAGTAT-CGTGAGTG,1.010000e+21,F2,...,female,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL
Ca006ONNL2,6,H2,SA508,GACACCGT,SB702,CGAAGTAT,SB702SA508,CGAAGTAT-GACACCGT,1.010000e+21,H2,...,female,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL


In [3]:
# Read in table at ASV level
biom_path = '../Data/Tables/Absolute_Abundance_Tables/feature_table_with_tax_labels_Genus.biom'
# biom_path = '../Data/Tables/Absolute_Abundance_Tables/209766_filtered_by_prevalence_1pct_rare_Genus-ASV-non-collapse.biom'
biom_tbl = load_table(biom_path)
df = pd.DataFrame(biom_tbl.to_dataframe().T)

# delete the prefix from the index
df.index = df.index.str.replace('15564.', '')

# Get samples that don't start with 'nares' in metadata
nares_samples = metadata[metadata['group'].str.startswith('nares')].index

# Subset to only certain area samples
nares_samples = metadata[metadata['area'].str.startswith('Cape Town')].index

# Filter df to keep only nares samples
df = df.loc[nares_samples]
df

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Bacillus_P_294101_ASV-2,g__Cutibacterium_ASV-1,g___ASV-18,g___ASV-28,g___ASV-25,...,g__Blautia_A_141781_ASV-7,g___ASV-358,g__Peptoniphilus_A_ASV-7,g___ASV-154,g___ASV-169,g__UBA952_ASV-1,g__Petroclostridium_ASV-1,g__Capnocytophaga_820690_ASV-4,g___ASV-202,g__Streptococcus_ASV-37
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,15.0,0,0,0,0,0,13.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ca010EBL,8.0,0,0,0,0,0,3.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
900051,30.0,0,0,0,0,0,10.0,5.0,0,2.0,...,0,0,0,0,0,0,0,0,0,0
900057,39.0,0,0,0,0,0,0,3.0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ca009STNL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca006ONL2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ca006ONNL,9.0,0,0,0,0,0,3.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ca006ONNL2,5.0,2.0,0,0,0,0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def rclr_transform(df, pseudocount=1e-6):
    """
    Applies Robust Centered Log-Ratio (RCLR) transformation to a DataFrame.
    Zeros are ignored in the geometric mean calculation per sample.
    """
    # Convert sparse to dense if needed
    df = df.sparse.to_dense()

    # Replace 0 with np.nan to ignore in log and mean
    df_masked = df.replace(0, np.nan)

    # Apply log (with pseudocount only where needed)
    log_df = np.log(df_masked + pseudocount)

    # Subtract mean of each row (feature)
    rclr_df = log_df.sub(log_df.mean(axis=1, skipna=True), axis=0)

    return rclr_df



rclr_df = rclr_transform(df)

In [5]:
# Map the 'group' column from metadata to df based on matching index
rclr_df['group'] = metadata.loc[rclr_df.index, 'group']
rclr_df

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Bacillus_P_294101_ASV-2,g__Cutibacterium_ASV-1,g___ASV-18,g___ASV-28,g___ASV-25,...,g___ASV-358,g__Peptoniphilus_A_ASV-7,g___ASV-154,g___ASV-169,g__UBA952_ASV-1,g__Petroclostridium_ASV-1,g__Capnocytophaga_820690_ASV-4,g___ASV-202,g__Streptococcus_ASV-37,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,0.713092,NaN,NaN,NaN,NaN,NaN,0.569991,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
Ca010EBL,0.803347,NaN,NaN,NaN,NaN,NaN,-0.177482,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
900051,2.021452,NaN,NaN,NaN,NaN,NaN,0.922840,0.229693,NaN,-0.686597,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
900057,2.170011,NaN,NaN,NaN,NaN,NaN,NaN,-0.394938,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
Ca009STNL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADNL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca006ONL2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
Ca006ONNL,0.729379,NaN,NaN,NaN,NaN,NaN,-0.369233,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADNL
Ca006ONNL2,0.437727,-0.478563,NaN,NaN,NaN,NaN,-1.171710,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADNL


In [6]:
# Filter for skin samples
nares_samples = rclr_df[rclr_df['group'].isin(['nares-AD', 'nares-H'])]
nares_samples

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Bacillus_P_294101_ASV-2,g__Cutibacterium_ASV-1,g___ASV-18,g___ASV-28,g___ASV-25,...,g___ASV-358,g__Peptoniphilus_A_ASV-7,g___ASV-154,g___ASV-169,g__UBA952_ASV-1,g__Petroclostridium_ASV-1,g__Capnocytophaga_820690_ASV-4,g___ASV-202,g__Streptococcus_ASV-37,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,2.148750,NaN,3.469198,NaN,NaN,NaN,NaN,-1.089928,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
Ca010EBPN,2.934590,NaN,2.498353,NaN,2.746189,NaN,0.642055,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900053,4.093420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900059,2.114945,1.846681,2.326254,2.143932,NaN,NaN,-0.312803,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
Co004LNPN,3.675128,NaN,2.936370,NaN,1.563321,NaN,0.564792,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Co003LTPN,2.689600,2.831916,1.505045,0.954999,-0.366757,NaN,-1.753051,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-H
900136,2.171598,2.336922,-0.371128,-0.707600,-0.237596,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-H
900126,3.974243,NaN,2.482588,NaN,NaN,NaN,-1.100930,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD


In [7]:
def pairwise_mannwhitney_bh(df, feature, group_col='group'):
    groups = df[group_col].unique()
    comparisons = list(itertools.combinations(groups, 2))

    raw_pvals = []
    labels = []
    pairs = []

    for g1, g2 in comparisons:
        x = df[df[group_col] == g1][feature].dropna()
        y = df[df[group_col] == g2][feature].dropna()
        _, p = mannwhitneyu(x, y, alternative='two-sided')
        raw_pvals.append(p)
        labels.append(f"{g1} vs {g2}")
        pairs.append((g1, g2))

    _, pvals_corrected, _, _ = multipletests(raw_pvals, method='fdr_bh')

    results_df = pd.DataFrame({
        'Comparison': labels,
        'Pair': pairs,
        'Raw p-value': raw_pvals,
        'BH-corrected p-value': pvals_corrected
    })

    return results_df

In [8]:
def pairwise_mannwhitney_bh(df, feature, group_col='group'):
    groups = df[group_col].unique()
    comparisons = list(itertools.combinations(groups, 2))

    raw_pvals = []
    labels = []
    pairs = []

    for g1, g2 in comparisons:
        x = df[df[group_col] == g1][feature].dropna()
        y = df[df[group_col] == g2][feature].dropna()

        # Skip comparison if either group has no data
        if len(x) == 0 or len(y) == 0:
            continue

        _, p = mannwhitneyu(x, y, alternative='two-sided')
        raw_pvals.append(p)
        labels.append(f"{g1} vs {g2}")
        pairs.append((g1, g2))

    if len(raw_pvals) == 0:
        return pd.DataFrame(columns=['Comparison', 'Pair', 'Raw p-value', 'BH-corrected p-value'])

    _, pvals_corrected, _, _ = multipletests(raw_pvals, method='fdr_bh')

    results_df = pd.DataFrame({
        'Comparison': labels,
        'Pair': pairs,
        'Raw p-value': raw_pvals,
        'BH-corrected p-value': pvals_corrected
    })

    return results_df


In [9]:
# Define custom color palette
group_palette = {
    'nares-H': '#ADD8E6',     # baby blue
    'nares-AD': '#E31A1C'    # red
}

strip_palette = {
    'nares-H': '#6CA6CD',     # darker baby blue
    'nares-AD': '#A50000'    # darker red
}


In [10]:
def plot_multi_taxa_boxplots(data, taxa_list, group_palette, strip_palette, title_name, order=['nares-AD', 'nares-H']):
    n_taxa = len(taxa_list)
    group_count = len(order)

    # Prepare long format dataframe
    plot_data = []
    for i, taxon in enumerate(taxa_list):
        for j, group in enumerate(order):
            xpos = i * group_count + j
            values = data.loc[data['group'] == group, taxon]
            for v in values:
                plot_data.append({
                    'x': xpos,
                    'taxon': taxon.replace(' g__', '').split('_')[0],
                    'value': v,
                    'group': group
                })

    plot_df = pd.DataFrame(plot_data)

    # Dynamically adjust figure width
    width_per_taxon = 3
    fig_width = max(5, min(n_taxa * width_per_taxon, 25))
    fig, ax = plt.subplots(figsize=(fig_width, 5))

    sns.boxplot(
        data=plot_df,
        x='x',
        y='value',
        hue='group',
        palette=group_palette,
        width=0.5,
        fliersize=0,
        dodge=False,
        ax=ax
    )

    sns.stripplot(
        data=plot_df,
        x='x',
        y='value',
        hue='group',
        palette=strip_palette,
        dodge=False,
        jitter=True,
        size=4,
        alpha=0.75,
        ax=ax
    )

    # Remove duplicated legends
    handles, labels = ax.get_legend_handles_labels()

    # Count samples per group
    group_counts = {g: data[data['group'] == g].shape[0] for g in order}

    # Create updated legend labels
    legend_labels = [f"{g.replace('skin-', '')} (n={group_counts[g]})" for g in order]

    # Dynamically adjust legend position
    legend_y_offset = -0.35
    ax.legend(
        handles[:3],
        legend_labels,
        loc='lower center',
        bbox_to_anchor=(0.5, legend_y_offset),
        ncol=len(order),
        fontsize=12,
        title_fontsize=10,
        frameon=True,
        borderaxespad=0
    )
    # ax.legend('')

    # X-tick labels per taxon group
    xtick_positions = [i * group_count + 1 for i in range(n_taxa)]
    xtick_labels = [taxon.replace('g__', '').replace('_ASV-', '\nASV-') for taxon in taxa_list]
    xtick_fontsize = 16 #if n_taxa > 2 else 16
    ax.set_xticks(xtick_positions)
    ax.set_xticklabels(xtick_labels, fontsize=xtick_fontsize)

    ax.set_xlim(-0.5, n_taxa * group_count - 0.5)
    ax.set_ylabel("RCLR-transformed abundance", fontsize=16)
    ax.tick_params(axis='y', labelsize=16)
    ax.set_xlabel(" ")

    ax.set_title(title_name, fontsize=18, pad=16)

    # ------------------------
    # Annotate p-values
    # ------------------------
    for i, taxon in enumerate(taxa_list):
        stats = pairwise_mannwhitney_bh(data, taxon)
        y_max = data[taxon].max()
        offset = y_max * 0.25

        for k, row in stats.iterrows():
            g1, g2 = row['Pair']
            pval = row['BH-corrected p-value']
            xpos1 = i * group_count + order.index(g1)
            xpos2 = i * group_count + order.index(g2)
            x = (xpos1 + xpos2) / 2
            y = y_max + offset * k

            stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
            label = f"{pval:.1e} {stars}"

            ax.plot([xpos1, xpos1, xpos2, xpos2], [y - 0.01, y, y, y - 0.01], lw=1, color='black')
            ax.text(x, y + 0.01, label, ha='center', va='bottom', fontsize=14)

    sns.despine()
    fig.subplots_adjust(bottom=0.25)
    return fig


In [11]:
taxa_list = ['g__Streptococcus_ASV-1', 'g__Streptococcus_ASV-2', 'g__Staphylococcus_ASV-1', 'g__Micrococcus_ASV-1', 'g__Veillonella_A_ASV-1']

fig = plot_multi_taxa_boxplots(nares_samples, taxa_list, group_palette, strip_palette, 'Differential Nares Taxa of Cape Town Samples')
fig.savefig('../Plots/Analysis_figures/RCLR_Abundance/rclr_multi_taxa_boxplot_all_taxa_ASV_nares_Cape Town.png', dpi=600)


/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


In [12]:
def plot_multi_taxa_boxplots(data, taxa_list, group_palette, strip_palette, title_name, order=['skin-H', 'skin-ADNL', 'skin-ADL']):
    n_taxa = len(taxa_list)
    group_count = len(order)

    # Prepare long format dataframe
    plot_data = []
    for i, taxon in enumerate(taxa_list):
        for j, group in enumerate(order):
            xpos = i * group_count + j
            values = data.loc[data['group'] == group, taxon]
            for v in values:
                plot_data.append({
                    'x': xpos,
                    'taxon': taxon.replace(' g__', '').split('_')[0],
                    'value': v,
                    'group': group
                })

    plot_df = pd.DataFrame(plot_data)

    # Dynamically adjust figure width
    width_per_taxon = 3
    fig_width = max(5, min(n_taxa * width_per_taxon, 25))
    fig, ax = plt.subplots(figsize=(fig_width, 5))

    sns.boxplot(
        data=plot_df,
        x='x',
        y='value',
        hue='group',
        palette=group_palette,
        width=0.5,
        fliersize=0,
        dodge=False,
        ax=ax
    )

    sns.stripplot(
        data=plot_df,
        x='x',
        y='value',
        hue='group',
        palette=strip_palette,
        dodge=False,
        jitter=True,
        size=4,
        alpha=0.75,
        ax=ax
    )

    # Remove duplicated legends
    handles, labels = ax.get_legend_handles_labels()

    # Count samples per group
    group_counts = {g: data[data['group'] == g].shape[0] for g in order}

    # Create updated legend labels
    legend_labels = [f"{g.replace('skin-', '')} (n={group_counts[g]})" for g in order]

    # Dynamically adjust legend position
    legend_y_offset = -0.35
    ax.legend(
        handles[:3],
        legend_labels,
        loc='lower center',
        bbox_to_anchor=(0.5, legend_y_offset),
        ncol=len(order),
        fontsize=12,
        title_fontsize=10,
        frameon=True,
        borderaxespad=0
    )
    # ax.legend('')

    # X-tick labels per taxon group
    xtick_positions = [i * group_count + 1 for i in range(n_taxa)]
    xtick_labels = [taxon.replace('g__', '').replace('_ASV-', '\nASV-') for taxon in taxa_list]
    xtick_fontsize = 16 #if n_taxa > 2 else 16
    ax.set_xticks(xtick_positions)
    ax.set_xticklabels(xtick_labels, fontsize=xtick_fontsize)

    ax.set_xlim(-0.5, n_taxa * group_count - 0.5)
    ax.set_ylabel("RCLR-transformed abundance", fontsize=16)
    ax.tick_params(axis='y', labelsize=16)
    ax.set_xlabel(" ")

    ax.set_title(title_name, fontsize=18, pad=16)

    # ------------------------
    # Annotate p-values
    # ------------------------
    for i, taxon in enumerate(taxa_list):
        stats = pairwise_mannwhitney_bh(data, taxon)
        y_max = data[taxon].max()
        offset = y_max * 0.25

        for k, row in stats.iterrows():
            g1, g2 = row['Pair']
            pval = row['BH-corrected p-value']
            xpos1 = i * group_count + order.index(g1)
            xpos2 = i * group_count + order.index(g2)
            x = (xpos1 + xpos2) / 2
            y = y_max + offset * k

            stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
            label = f"{pval:.1e} {stars}"

            ax.plot([xpos1, xpos1, xpos2, xpos2], [y - 0.01, y, y, y - 0.01], lw=1, color='black')
            ax.text(x, y + 0.01, label, ha='center', va='bottom', fontsize=14)

    sns.despine()
    fig.subplots_adjust(bottom=0.25)
    return fig


In [13]:
taxa_list = ['g__Streptococcus_ASV-1', 'g__Streptococcus_ASV-2', 'g__Staphylococcus_ASV-1']

fig = plot_multi_taxa_boxplots(nares_samples, taxa_list, group_palette, strip_palette, 'Differential Skin Taxa of Cape Town Samples')
fig.savefig('../Plots/Analysis_figures/RCLR_Abundance/rclr_multi_taxa_boxplot_all_taxa_ASV_skin_Cape Town.png', dpi=600)


ValueError: Could not interpret input 'x'